# Recommendation System

Recommendation system have emerged as an increasingly important tool for ecommerce, entertainment, and social media. Users are inundated by the number of choices available e.g. while shopping on *Amazon*, or watching movies on *Netflix*, thus a good recommendation system can make a huge difference in user experience. 

Over 80% of the content watched on Netflix today is powered by their recommendation model. Simlarly, Amazon, YouTube, Hulu, Google News - all report improvement in engagement metrics when they deploy recommendation systems. Every year a major scientific conference, **RecSys**, is organized where research at the frontier of recommendation systems - both in academia and industry - is presented. 

In this post, we will discuss statergies to build a simple recommendation engine. Our discussion follows the dictum - 'People Who Like A Also Like B', which is the basis of **Collaborative Filtering Recommendation systems**. 

## Data 

We will use the **MovieLens** data which contains ratings of movies by users. The data which is publically available from [MovieLens Website](https://grouplens.org/datasets/movielens/). We are using 100k data which has 100k ratings. 

In [1]:
import pandas as pd
import numpy as np

In [5]:
data_path = '../data/'

In [6]:
rating_df = pd.read_csv(data_path + 'ratings.csv', sep=',', header=0)

In [7]:
rating_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
rating_df.shape

(100836, 4)

In [9]:
rating_df.userId.nunique()

610

In [10]:
rating_df.movieId.nunique()

9724

- The data has over 100k ratings by 610 users on 9724 movies

In [11]:
max(rating_df.userId), max(rating_df.movieId)

(610, 193609)

  - The movie ids do not follow an order. 
  - 9724 movies have been selected that users with id 1 to 610 have rated so as to have 100k ratings. 

### Movies Information

We are also provided the titles and genres of the movies in a separate file. 

In [12]:
movie_df = pd.read_csv(data_path + 'movies.csv', sep=',', header=0)

In [13]:
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
movie_df[movie_df['title'].str.contains('Avengers')]

,movieId,title,genres
1611,2153,"Avengers, The (1998)",Action|Adventure
6148,44020,Ultimate Avengers (2006),Action|Animation|Children|Sci-Fi
7693,89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX
8551,115727,Crippled Avengers (Can que) (Return of the 5 D...,Action|Adventure
8686,122892,Avengers: Age of Ultron (2015),Action|Adventure|Sci-Fi
8693,122912,Avengers: Infinity War - Part I (2018),Action|Adventure|Sci-Fi
9153,147657,Masked Avengers (1981),Action
9488,170297,Ultimate Avengers 2 (2006),Action|Animation|Sci-Fi


## Neighborhood Methods

### Set Similarity
If we ignore the ratings that the users have given to the movies, and consider the movies that the users have watched, we get a set of movies/users for every user/movie. Think of this formulation as a bipartite graph of users and movies where there is an edge between a user and a movie if a user has watched the movie, the edges have all same weights. 

Create a dictionary of movies as keys and values are users that have rated them

In [15]:
movie_sets = dict((movie, set(users)) for movie, users in rating_df.groupby('movieId')['userId'])

Since we have a set of users to characterize each movie, to compute the similarity of two movies, we use **Jaccard Index** which, for two sets, is the ratio of number of elements in the intersection and number of elements in the union.

In [16]:
def jaccard(movie1, movie2, movie_sets):
    a = movie_sets[movie1]
    b = movie_sets[movie2]
    intersection = float(len(a.intersection(b)))
    return intersection / (len(a) + len(b) - intersection)

Let's explore similarity between some movies, qualitatively. We use the movies dataframe to get the names of the movies via their Ids.  

In [68]:
movie_df[movie_df.movieId == 260].title.values[0]

'Star Wars: Episode IV - A New Hope (1977)'

In [67]:
print(movie_df[movie_df.movieId == 1196].title.values[0])
print(jaccard(260, 1196, movie_sets))

Star Wars: Episode V - The Empire Strikes Back (1980)
0.6985294117647058


In [69]:
print(movie_df[movie_df.movieId == 1210].title.values[0])
print(jaccard(260, 1210, movie_sets))

Star Wars: Episode VI - Return of the Jedi (1983)
0.6433823529411765


In [66]:
print(movie_df[movie_df.movieId == 1].title.values[0])
print(jaccard(260, 1, movie_sets))

Toy Story (1995)
0.4036144578313253


The movie Star Wars IV has higher similarity score with other Star Wars as compared to Toy Story. 

Using the Jaccard Index, we can retrieve top-k similar movies to a given movie. This provides a way to recommend movies of a user which are similar to the movies that the user has watched. 

In [23]:
import operator 

def get_similar_movies_jaccard(movieid, movie_sets, top_n=5):
    movie = movie_df[movie_df.movieId == movieid].title.values[0]
    jaccard_dict = {x: jaccard(x, movieid, movie_sets) for x in movie_sets}
    ranked_movies = sorted(jaccard_dict.items(), key=operator.itemgetter(1), reverse=True)[:top_n]
    sim_movies = [movie_df[movie_df.movieId == id[0]].title.values[0] for id in ranked_movies]
    return {'movie': movie, 'sim_movies': sim_movies}
    

In [24]:
get_similar_movies_jaccard(260, movie_sets)

{'movie': 'Star Wars: Episode IV - A New Hope (1977)',
 'sim_movies': ['Star Wars: Episode IV - A New Hope (1977)',
  'Star Wars: Episode V - The Empire Strikes Back (1980)',
  'Star Wars: Episode VI - Return of the Jedi (1983)',
  'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
  'Matrix, The (1999)']}

In [25]:
get_similar_movies_jaccard(1, movie_sets)

{'movie': 'Toy Story (1995)',
 'sim_movies': ['Toy Story (1995)',
  'Independence Day (a.k.a. ID4) (1996)',
  'Jurassic Park (1993)',
  'Star Wars: Episode IV - A New Hope (1977)',
  'Forrest Gump (1994)']}

### Vector Similarity

Rather than the set based similarity like Jaccard, we can define every movie as a sparse vector of dimension equal to the number of users and the vector entry corresponding to each user is given by the rating that the user has for the movie or zero if no rating exists (i.e. the user hasn't seen/rated the movie). 

In [36]:
from scipy.spatial.distance import cosine

In [37]:
num_users = rating_df.userId.nunique()
num_users

610

In [28]:
movie_sparse_vecs = []
movies = []
for movie, group in rating_df.groupby('movieId'):
    vec = [0] * num_users
    for x in group[['userId', 'rating']].values:
        vec[int(x[0]) - 1] = x[1]
    movie_sparse_vecs.append(vec)
    movies.append(movie)

In [29]:
len(movie_sparse_vecs), len(movies)

(9724, 9724)

In [30]:
movie_sparse_vecs = np.array(movie_sparse_vecs)

In [31]:
movie_sparse_vecs.shape

(9724, 610)

In [32]:
movie_sparse_vecs[1].shape

(610,)

In [33]:
movie2id = {x:i for i,x in enumerate(movies)}

In [34]:
movie2id[260], movie2id[1196]

(224, 897)

In [38]:
1.0 - cosine(movie_sparse_vecs[224], movie_sparse_vecs[897])

0.8324073552233735

In [39]:
movie2id[1210], 1.0 - cosine(movie_sparse_vecs[224], movie_sparse_vecs[910])

(910, 0.7906385655616323)

In [40]:
1.0 - cosine(movie_sparse_vecs[224], movie_sparse_vecs[0])

0.5573881705799366

In [41]:
def get_similar_movies_nbd_cosine(movieid, movie_vecs, top_n=5):
    movie = movie_df[movie_df.movieId == movieid].title.values[0]
    movie_idx = movie2id[movieid]
    query = movie_vecs[movie_idx].reshape(1,-1)
    ranking = cosine_similarity(movie_vecs,query)
    top_ids = np.argsort(ranking, axis=0)
    top_ids = top_ids[::-1][:top_n]
    top_movie_ids = [movies[j[0]] for j in top_ids]
    sim_movies = [movie_df[movie_df.movieId == id].title.values[0] for id in top_movie_ids]
    return {'movie': movie, 'sim_movies': sim_movies}

In [42]:
movieid = 1
movie_data = movie_sparse_vecs
get_similar_movies_nbd_cosine(movieid, movie_data, top_n=5)

{'movie': 'Toy Story (1995)',
 'sim_movies': ['Toy Story (1995)',
  'Toy Story 2 (1999)',
  'Jurassic Park (1993)',
  'Independence Day (a.k.a. ID4) (1996)',
  'Star Wars: Episode IV - A New Hope (1977)']}

In [43]:
movieid = 260
movie_data = movie_sparse_vecs
get_similar_movies_nbd_cosine(movieid, movie_data, top_n=5)

{'movie': 'Star Wars: Episode IV - A New Hope (1977)',
 'sim_movies': ['Star Wars: Episode IV - A New Hope (1977)',
  'Star Wars: Episode V - The Empire Strikes Back (1980)',
  'Star Wars: Episode VI - Return of the Jedi (1983)',
  'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
  'Matrix, The (1999)']}

## Latent Factor Model

A very popular technique for recommendation systems is via matrix factorization. The idea is to reduce the dimensionality of the data before calculating similar movies/users. We factorize the user-item matrix to obtain the user factors and item factors which are the low-dimensional embeddings such that 'similar' user/items are mapped to 'nearby' points.  

This kind of analysis can generate matches that are impossible to find with the techniques discussed above as the latent factors can capture attributes which are hard for raw data to deciper e.g. a latent factor can correspond to the degree to which a movie is female oriented or degree to which there is a slow development of the charcters. 

Moreover, the user and the movies are embedded to the same space, which provides a direct way to compute user-movie similarity.  

Create a matrix of ratings. 

In [44]:
ratings_mat = np.ndarray(
    shape=(np.max(rating_df.movieId.values), np.max(rating_df.userId.values)),
    dtype=np.uint8)
ratings_mat[rating_df.movieId.values-1, rating_df.userId.values-1] = rating_df.rating.values

In [45]:
ratings_mat.shape

(193609, 610)

Normalize the rating matrix

In [46]:
normalised_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, 1))]).T

We will use **Singular Value Decomposition (SVD)** for factorizing the matrix. I'm delibately skipping any discussion of the mathematics of SVD. Since the user-movie rating matrix is very sparse, it is more efficient to use the implementation from scipy.sparse. 

In [47]:
from scipy.sparse import coo_matrix, csr_matrix
from scipy.sparse.linalg import svds, norm

The number of the latent-factors is chosen to be 50 i.e. top-50 singular values of the SVD are considered. The choice of the number of latent factors is a hyperparameter of the model, and requires a more sophisticated analysis to tune. We provide no reason for the choice of 50. 

In [48]:
n_factors = 50

In [49]:
A = normalised_mat.T / np.sqrt(ratings_mat.shape[0] - 1)
U, S, V = svds(A, n_factors)

In [50]:
U.shape

(610, 50)

In [51]:
V.shape

(50, 193609)

In [52]:
movie_factors = V.T
user_factors = U

Instead of representing each movie as a sparse vector of the ratings of all 360,000 possible users for it, after factorizing the matrix each movie will be represented by a 50 dimensional dense vector.

Let's study some examples to have a qualitative understanding. Cosine similarity of the latent factors of two movies signifies how similar the movies are.

In [53]:
idx = 260
movie_df[movie_df.movieId == idx].title.values[0],  movie_df[movie_df.movieId == 1196].title.values[0]

('Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)')

In [54]:
1.0 - cosine(movie_factors[259], movie_factors[1195])

0.8777832979913568

In [55]:
movie_df[movie_df.movieId == 1210].title.values[0], 1.0 - cosine(movie_factors[259], movie_factors[1209])

('Star Wars: Episode VI - Return of the Jedi (1983)', 0.8518636866885215)

In [56]:
movie_df[movie_df.movieId == 1].title.values[0], 1.0 - cosine(movie_factors[259], movie_factors[0])

('Toy Story (1995)', 0.20152844265131886)

The similarity of the 'Star Wars: Episode IV - A New Hope' is higher for the movies 'Star Wars: Episode V - The Empire Strikes Back' and 'Star Wars: Episode VI - Return of the Jedi' and is much lower for 'Toy Story'. Moreover, the 'Star Wars: Episode VI' is closer to 'Star Wars: Episode IV' than the 'Star Wars: Episode V'.  

Define a routine to get top-n movies similar to a given movie. 

In [57]:
def get_similar_movies_matrix_factorization(data, movieid, top_n=10):
    index = movieid - 1 # Movie id starts from 1
    movie = movie_df[movie_df.movieId == movieid].title.values[0]
    movie_row = data[index].reshape(1,-1)
    similarity = cosine_similarity(movie_row, data)
    sort_indexes = np.argsort(-similarity)[0]
    return {'movie': movie, 'sim_movies': [movie_df[movie_df.movieId == id].title.values[0] for id in sort_indexes[:top_n] + 1]}

In [58]:
movie_id = 260
get_similar_movies_matrix_factorization(movie_factors, movie_id)

{'movie': 'Star Wars: Episode IV - A New Hope (1977)',
 'sim_movies': ['Star Wars: Episode IV - A New Hope (1977)',
  'Star Wars: Episode V - The Empire Strikes Back (1980)',
  'Star Wars: Episode VI - Return of the Jedi (1983)',
  'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
  'Lesson Faust (1994)',
  'Touch (1997)',
  'Inferno (2016)',
  'Beverly Hills Chihuahua (2008)',
  'Matrix, The (1999)',
  'Star Wars: Episode III - Revenge of the Sith (2005)']}

In [59]:
movie_id = 1
get_similar_movies_matrix_factorization(movie_factors, movie_id)

{'movie': 'Toy Story (1995)',
 'sim_movies': ['Toy Story (1995)',
  'Toy Story 2 (1999)',
  'Adventures of Pinocchio, The (1996)',
  'Eddie (1996)',
  'Children of the Corn IV: The Gathering (1996)',
  'Twister (1996)',
  'Sudden Death (1995)',
  'Dear God (1996)',
  'Kazaam (1996)',
  'Sunset Park (1996)']}

In [60]:
user_factors.shape, movie_factors.shape

((610, 50), (193609, 50))

Since the user and movies are in the same space, we can also compute movies similar to a user. A recommendation model can be defined as showing movies similar to the given user.  

In [61]:
def get_recommendations_matrix_factorization(userid, user_factors, movie_factors, top_n=10):
    user_vec = user_factors[userid - 1].reshape(1,-1)
    similarity = cosine_similarity(user_vec, movie_factors)
    sort_indexes = np.argsort(-similarity)[0]
    return [movie_df[movie_df.movieId == id].title.values[0] for id in sort_indexes[:top_n] + 1]   

In [62]:
top_recos = get_recommendations_matrix_factorization(1, user_factors, movie_factors)
top_recos

['Best Men (1997)',
 "Gulliver's Travels (1939)",
 'Newton Boys, The (1998)',
 'Teenage Mutant Ninja Turtles III (1993)',
 'Welcome to Woop-Woop (1997)',
 'Romancing the Stone (1984)',
 'Wolf Man, The (1941)',
 'Red Dawn (1984)',
 'Goonies, The (1985)',
 'Howard the Duck (1986)']

This concludes our discussion on collaborative filtering based recommendation system. We discussed neighborhood based similarity and latent factor based similarity. The treatment was kept simple, without any mathematical details.  Further, there is no discussion on evaluation of the recommendation systems. Only qualitative evaluation, based on movie-movie similarity, is presented. Ideally, we would create train and test sets and use the test set to evaluate the model via metrics e.g. *Mean Average Precision*. 